In this file, we will try to see if the full moon as an influence over the birth number in France. 
To do so, we found a dataset on the INSEE website containing the birth number day by day in France from 1968 to 2018.

In [1]:
import pandas as pd

data = pd.read_excel("T79JNAIS.xls")
processed = pd.DataFrame(columns=["Year","Month","Day","Birth"])
data

,T79JNAIS : Répartition quotidienne des naissances vivantes,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33
0,CHAMP : France métropolitaine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,JOUR,Total du mois,1.0,2.0,3.0,4.0,5.0,6.0,7.0,...,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29,30,31
3,Année de 1968 à 2018,Mois,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1968,Janvier,70409,2080.0,2319.0,2335.0,2338.0,2319.0,2212.0,2083.0,...,2268.0,2290.0,2345.0,2254.0,2324.0,2303.0,2232.0,2335,2352,2303
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
612,NaN,Septembre,61834,1788.0,1761.0,1926.0,2175.0,2188.0,2147.0,2101.0,...,1940.0,1892.0,2145.0,2176.0,2229.0,2257.0,2219.0,1973,1875,so
613,NaN,Octobre,63588,2184.0,2238.0,2188.0,2208.0,2200.0,1912.0,1803.0,...,2023.0,2082.0,2174.0,1988.0,2155.0,1821.0,1893.0,2015,2155,2144
614,NaN,Novembre,59937,1849.0,1931.0,1796.0,1830.0,2173.0,2157.0,2143.0,...,2061.0,2098.0,1864.0,1743.0,2130.0,2113.0,2032.0,1983,2028,so
615,NaN,Décembre,59931,1838.0,1729.0,2006.0,2061.0,1997.0,2019.0,2049.0,...,1764.0,1592.0,1770.0,1551.0,1918.0,2193.0,2127.0,1719,1742,1881


The next step is to extract the data and store them in a more friendly format.
We first create tabs, one for each value we want to extract.

In [2]:
years = []
months = []
days = []
births = []

year = -1
month = -1

for i in range(4,604,12):
    if not(data.iat[i,0]== "NaN") :
        year = data.iat[i,0]
        for j in range(0,12):
            month = data.iat[i+j,1]
            for k in range(0,31):
                if not data.iat[i+j,k+3]=="so":
                    years.append(year)
                    months.append(month)
                    days.append(k+1)
                    births.append(data.iat[i+j,k+3])
                    
    
        

Then we create a DataFrame, a commun data structure which allows easy processing over the data

In [3]:
d = {'year':years,'month':months,'day':days,'birth':births}
processed = pd.DataFrame(d)

Now, a line in the DataFrame correspond to one day.
The fime period between two full moon is 29,53 days.
Our next step is to create a tab with only the birth numbers when it's a full moon

In [4]:
moonTab = []
period = 29.53
reference = 0
moon = 0
moonTotal = 0
for i in range(processed.shape[0]):
    reference += processed.at[i,'birth']
    if i == round(period * moonTotal,0) + 14:
        moonTotal += 1
        moon += processed.at[i,'birth']
        moonTab.append(processed.at[i,'birth'])


In order to compare our two series we choose to compare the means

In [5]:
ReferenceMean = reference/18263
MoonMean = moon/moonTotal


The average of birth from 1968 to 2018 is 2121 a day

In [6]:
ReferenceMean

2121.3138586212563

The average of birth from 1968 to 2018 is 2131 when there is a full moon

In [7]:
MoonMean

2130.535598705502

So it seems that there are very little more birth when the moon is full 

In order to test our hypothesys, we do a Student test

Note : we are not sure the following sections means anything as we are not sure it is the right test to do.

H0 is the two means are equals, the confidence of the test will be 95%

In [8]:
import numpy

ReferenceStd = numpy.std(processed.at[i,'birth'])
moonStd = numpy.std(moonTab)

In [9]:
from scipy import stats
e = stats.ttest_ind_from_stats(ReferenceMean,ReferenceStd,processed.shape[0],MoonMean,moonStd,moonTotal)

In [10]:
e

Ttest_indResult(statistic=-5.118327838015737, pvalue=3.1128066797743754e-07)

The pvalue is below 0.05 so the reject our hypothesys.

The number of birth on full moon days is significantly higher than standard days